In [ ]:
import pandas as pd
import numpy as np
from custom_detrending import custom_detrending, refine_detrended_flux_err, iteratively_remove_sines

import matplotlib.pyplot as plt
%matplotlib inline

from altaipony.lcio import from_path

from copy import deepcopy

from k2sc.standalone import detrend

In [ ]:
EPIC = 211970427
c = 18


In [ ]:
#for i in range(12,37):
I=37
flcd = from_path(f"../DetrendedLightCurves/SC_{EPIC}/{I}_{EPIC}_{c}.fits", mission="K2", mode="AltaiPony")
#flcd = from_path(f"/home/ekaterina/Downloads/0_211970427_18.fits", mission="K2", mode="AltaiPony")
#flcd = custom_detrending(flcd)

plt.figure(figsize=(15,5))
plt.plot(flcd.time, flcd.detrended_flux)
#plt.plot(flc.time, flc.flux)
plt.plot(flcd.time, flcd.flux+1000, c="r");
#plt.plot(flcd.time, flcd.detrended_flux+3*flcd.detrended_flux_err, c="k");
#plt.xlim(3428.5,3428.6)
plt.ylim(12000, 13000)
flcd.detrended_flux_err[:] = np.nanmedian(flcd.detrended_flux_err)

flares = flcd.find_flares().flares
flares

In [ ]:
%matplotlib inline
for i, r in flares.iterrows():
    plt.figure(figsize=(7,5))
    plt.plot(flcd.time, flcd.detrended_flux,c="k")
    plt.plot(flcd.time, flcd.flux,c="b", alpha=.3)
    sta, sto = int(r.istart), int(r.istop)
    plt.scatter(flcd.time[sta:sto], flcd.detrended_flux[sta:sto], c="r", label=r.cstart)#, label=f"EPIC {EPIC}, C{C}")#
    plt.xlim(r.tstart-.1,r.tstop+.1)
    #plt.ylim(np.nanmin(1.*flcd.detrended_flux),1.*np.nanmax(flcd.detrended_flux))
    plt.ylim(12000, 13000)
    plt.legend()

In [ ]:
flares = pd.DataFrame(flares.loc[[2,3],:])
flares

In [ ]:
with open("../k2scoc/results/tables/SC_flares.csv", "a") as f:
    flares["C"] = c
    flares["EPIC"] = EPIC
    flares["i"] = I
    flares.to_csv(f,index=False,header=False )

In [ ]:
flares = flares.T
flares